# Adding Dependeces

In [58]:
using DelimitedFiles
using CUDA
using ProgressMeter
include("../init/forces.jl")
include("../init/Neighbor_CUDA.jl")

knn_cu (generic function with 1 method)

# Initial Parameters

In [59]:
# Physical Conditions
const R_agg = 10
const t_f = 10000

# Time model Conditions
dt = 0.5
n_knn = 25

# Constants for Force Model
r_max, s = 2.5, 1.8
K = 1
nn = 16

16

# Functions

In [60]:
function init()
    # Initial Coordinates of Aggregates
    global X = Float32.(readdlm("../data/Initial/XYZ/Init/Sphere/$R_agg.xyz")[3:end,2:end]) |> cu

    # Inizializate Variables for kNN
    global i_Cell = CuArray{Float32}(undef, (size(X, 1), size(X, 1), 3))
    global Dist = CuArray{Float32}(undef, (size(X, 1), size(X, 1)))
    global idx = hcat([[CartesianIndex(i,1) for i=1:nn] for j=1:size(X,1)]...) |> cu

    # Inizializate Variables for Forces
    global r = zeros(nn,size(X)[1],3) |> cu
    global dist = zeros(nn, size(X)[1]) |> cu
    global F = zeros(nn, size(X)[1],3) |> cu
    global dX = zeros(size(X)[1],3) |> cu;
end

function init_2()
    # Initial Coordinates of Aggregates
    global X = Float32.(readdlm("../data/Initial/XYZ/Init/Sphere/$R_agg.xyz")[3:end,2:end]) |> cu

    # Inizializate Variables for kNN
    global i_Cell = CuArray{Float32}(undef, (size(X, 1), size(X, 1), 3))
    global Dist = CuArray{Float32}(undef, (size(X, 1), size(X, 1)))
    global idx = hcat([[CartesianIndex(i,1) for i=1:nn] for j=1:size(X,1)]...) |> cu

    # Inizializate Variables for Forces
    global r = zeros(nn,size(X)[1],3) |> cu
    global dist = zeros(nn, size(X)[1]) |> cu
    global F = zeros(nn, size(X)[1],3) |> cu
    global dX = zeros(size(X)[1],3) |> cu;
end

init (generic function with 1 method)

In [61]:
function main(n_knn)
    println("------------Init()-----------")
    init()

    p = Progress(Int(t_f/dt),barlen=25)
    for i in 0:Int(t_f/dt)

        if mod(i, n_knn) == 0
            # Calculating kNN
            knn_cu()
        end

        # Calculating Forces
        forces_cu()

        global X = X + dX*dt
        next!(p)
    end
end
function fusion(n_knn)
    println("------------Init()-----------")
    init()

    p = Progress(Int(t_f/dt),barlen=25)
    for i in 0:Int(t_f/dt)

        if mod(i, n_knn) == 0
            # Calculating kNN
            knn_cu()
        end

        # Calculating Forces
        forces_cu()

        global X = X + dX*dt
        next!(p)
    end
end
main(n_knn)

main (generic function with 1 method)

# Fusioning tho Spheres

In [65]:
Size = Float32((findmax(X[:,1])[1] - findmin(X[:,1])[1])/2 + 1)
X_2 = vcat(X,X) |> cu
X_2[1:size(X,1),1] = X_2[1:size(X,1),1] .- (Size)
X_2[size(X,1):end,1] = X_2[size(X,1):end,1] .+ Size
X_2 = Matrix(X_2)

1492×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -1.34397   -2.59802   -8.08334
  0.456032  -2.59802   -8.08334
  2.25603   -2.59802   -8.08334
 -2.24397   -1.03917   -8.08332
 -0.443968  -1.03917   -8.08332
  1.35603   -1.03917   -8.08332
  3.15603   -1.03917   -8.08332
 -3.14397    0.519676  -8.08331
 -1.34397    0.519676  -8.08331
  0.456032   0.519676  -8.08331
  ⋮                    
  0.456032  -0.519676   8.08331
  2.25603   -0.519676   8.08331
 -2.24397    1.03917    8.08332
 -0.443968   1.03917    8.08332
  1.35603    1.03917    8.08332
  3.15603    1.03917    8.08332
 -1.34397    2.59802    8.08334
  0.456032   2.59802    8.08334
  2.25603    2.59802    8.08334

In [68]:
X_f = zeros(size(X,1))
X = Matrix(X)
for i in 1:size(X_f,1)
    A = sqrt(X[i,1]^2+X[i,2]^2+X[i,3]^2)
    if A > 0.7*R_agg
        X_f[i] = 1
    else
        X_f[i] = 2
    end
end
X_f = Int.(vcat(X_f,X_f));

In [69]:
@time open("Test_Init.xyz"; write=true) do f
    write(f, "$(size(X_2, 1))\n")
    write(f, "Initial Data ($(R_agg))\n")
    writedlm(f,hcat(X_f, X_2), ' ')
end

  0.065177 seconds (94.22 k allocations: 5.828 MiB, 98.09% compilation time)


# Running with Two

In [80]:
X = X_2 |> cu

1492×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -1.5  -2.89  -8.98
  0.5  -2.89  -8.98
  2.5  -2.89  -8.98
 -2.5  -1.15  -8.98
 -0.5  -1.15  -8.98
  1.5  -1.15  -8.98
  3.5  -1.15  -8.98
 -3.5   0.58  -8.98
 -1.5   0.58  -8.98
  0.5   0.58  -8.98
  ⋮           
  0.5  -0.58   8.98
  2.5  -0.58   8.98
 -2.5   1.15   8.98
 -0.5   1.15   8.98
  1.5   1.15   8.98
  3.5   1.15   8.98
 -1.5   2.89   8.98
  0.5   2.89   8.98
  2.5   2.89   8.98

In [83]:
main(n_knn)

------------Init()-----------


Progress:   0%|                         |  ETA: 0:01:27

Progress:   0%|▏                        |  ETA: 0:01:27

Progress:   1%|▏                        |  ETA: 0:01:21

Progress:   1%|▏                        |  ETA: 0:01:46

...

Progress:  99%|████████████████████████▉|  ETA: 0:00:01

Progress: 100%|████████████████████████▉|  ETA: 0:00:00

Progress: 100%|████████████████████████▉|  ETA: 0:00:00

Progress: 100%|█████████████████████████|  ETA: 0:00:00

Progress: 100%|█████████████████████████| Time: 0:01:29


In [84]:
X

746×3 CuArray{Float64, 2, CUDA.Mem.DeviceBuffer}:
 -1.34397   -2.59801   -8.08334
  0.456032  -2.59801   -8.08334
  2.25603   -2.59801   -8.08334
 -2.24397   -1.03917   -8.08332
 -0.443968  -1.03917   -8.08332
  1.35603   -1.03917   -8.08332
  3.15603   -1.03917   -8.08332
 -3.14397    0.519676  -8.08331
 -1.34397    0.519676  -8.08331
  0.456032   0.519676  -8.08331
  ⋮                    
  0.456032  -0.519676   8.08331
  2.25603   -0.519676   8.08331
 -2.24397    1.03917    8.08332
 -0.443968   1.03917    8.08332
  1.35603    1.03917    8.08332
  3.15603    1.03917    8.08332
 -1.34397    2.59801    8.08334
  0.456032   2.59801    8.08334
  2.25603    2.59801    8.08334

In [75]:
@time open("Test_Final.xyz"; write=true) do f
    write(f, "$(size(X, 1))\n")
    write(f, "Initial Data ($(R_agg))\n")
    writedlm(f,hcat(X_f, X), ' ')
end

ArgumentError: ArgumentError: number of rows of each array must match (got (1492, 746))

In [70]:
run(`paraview Test_N.xyz`)

Process(`paraview Test_N.xyz`, ProcessExited(0))